# Manuscript Utils
This notebook contains useful functions for generating figures in the paper.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

## Expected Plotting
Stacked bar plots with tabulated relative abundances.

In [ ]:
def get_all_expected(root_dir: str, rank="Genus") -> pd.DataFrame:
    combined_expected = pd.DataFrame()
    for root, dirs, files in os.walk(root_dir):
        for f in files:
            # print("files: ", files)
            if f"expected_{rank.lower()}_annotated" in f and f.endswith(".csv"):
                # print(f)
                df = pd.read_csv(os.path.join(root, f), index_col=0, names=[rank, 'RA', "TAX_ID"], header=0)
                df["Source"] = root.split("/")[-1]

                # Files are of s#_expected.csv, so we can split on the underscore and take the first part.
                df["SampleID"] = f.split("_")[0]
                combined_expected = pd.concat([combined_expected, df], axis=0)

    return combined_expected

expected = get_all_expected("../../pipelines/", rank="Genus")
display(expected.head())
# Print the number of occurences in Source
expected["Source"].value_counts()

In [ ]:
for source, source_df in expected.groupby("Source"):
    for sample, sample_df in source_df.groupby("SampleID"):
        plot_df = sample_df.loc[sample_df["RA"] > 0]

        plot_df[["RA"]].T.plot(kind="barh", stacked=True, figsize=(20, 10))
        plt.show()

        plt.pie(plot_df["RA"], labels=plot_df.index, autopct='%1.1f%%')
        plt.show()

        # sns.barplot(x='SampleID', y='RA', hue="Genus", data=plot_df)
        # plt.show()